In [ ]:
import pandas as pd
import deepsmiles
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import random
import torch


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### DATA SET

In [ ]:
path="../BD/DeepSmile.csv"
df=df = pd.read_csv(path, sep=",")

converter = deepsmiles.Converter(rings=True, branches=True)

In [ ]:
deep_SMILES = []
for i in range(len(df)):
  deep_smile = converter.encode(df.smiles[i])
  deep_SMILES.append(deep_smile)

df["DeepSmiles"] = deep_SMILES

In [ ]:
df.to_csv("../DeepSmiles.csv",index=False)
path = "../DeepSmiles.csv"

#### CREACION DE VECTORES CARACTERISTICOS

In [ ]:
path = "../BD/DeepSmiles.csv"
df = pd.read_csv(path, sep=",")


In [ ]:
max = 0
for i in range(len(df)):
  if len(df.DeepSmiles[i]) > max:
    max = len(df.DeepSmiles[i])
print(max)

In [ ]:
x = set(list(' '.join(df.smiles.values)))
keys = []
for char in x:
  keys.append(char)

OHE_smiles = []

for smile in df.smiles.values[0:5000]:
  matrix = np.zeros((264,34)) #34 is the number of unique characters, 254 is the number of the max length of the SMILES string
  for count,char in enumerate(smile):
    for count2,key in enumerate(keys):
      if key == char:
        matrix[count][count2] = 1
  OHE_smiles.append(matrix.T)

  input_shape = (264, 34)

### CNN-BATCH-TRAIN-TEST


In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer1 =  nn.Sequential(
        nn.Conv1d(in_channels = 34 , out_channels = 32, kernel_size = 3, stride = 1, padding = 2),
        nn.ReLU(),
        nn.Conv1d(in_channels =  32 , out_channels = 16, kernel_size = 3, stride = 1, padding = 2),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2, stride=2))
    self.fc =  nn.Sequential(
        nn.Linear(2144, 1000),
        nn.ReLU(),
        nn.Linear(1000, 50),
        nn.ReLU(),
        nn.Linear(50, 1),
        nn.Sigmoid()
        )

  def forward(self, x):
    out = self.layer1(x)
    out = out.reshape(out.size(0), -1)
    output = self.fc(out)
    return output





def batch(x_train, y_train, batch_number):
  model.eval()
  with torch.no_grad():

    list1 = range(len(x_train))
    n_batch = random.sample(list1, batch_number)
    x_batch =  x_train[n_batch,:,:]
    y_batch = y_train[n_batch]

  return x_batch, y_batch



def train(model, funcion, optimizer,epochs, x_train, y_train, x_test, y_test, batch_number):
  loss_lista_train = []
  loss_lista_test = []
  i=0

  for epoch in range(epochs):

    x_batch, y_batch = batch(x_train, y_train, batch_number)


    x_batch = x_batch.to(device)
    y_pred =   model(x_batch)
    y_batch = y_batch.to(device)
    loss = funcion(y_pred,y_batch.unsqueeze(1))
    loss_lista_train.append(loss.cpu().detach().numpy())
    i=i+1

    model.eval()
    with torch.no_grad():
      x_test = x_test.to(device)
      y_pred_test =   model(x_test)
      y_test = y_test.to(device)
      loss_test = funcion(y_pred_test,y_test.unsqueeze(1))
      loss_lista_test.append(loss_test.cpu().detach().numpy())



    if (epoch%10==0):
      print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    model.train()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch%10==0):
      test(model, x_test, y_test)

  return loss_lista_train, loss_lista_test

def test(model, x_test, y_test):
  x_test = x_test.to(device)
  y_test = y_test.to(device)
  y_pred =   model(x_test)
  predictions_label = [1 if y_pred[i] >= 0.5 else 0 for i in range(len(y_pred))]
  pred = torch.Tensor(predictions_label)
  pred = pred.to(device)
  Com = torch.eq(y_test,pred)
  accu = sum(Com.long())/len(Com)
  print(accu)
  return


####  Definición de datos de entrada y datos de salida deseados

In [ ]:
x = np.array(OHE_smiles)
y = np.array(df.PPARD.values[0:10000])
y_class = np.array([])
for item in y:
  if item <= -10.0:
    y_class = np.append(y_class,1)
  else:
    y_class = np.append(y_class,0)

x_train = x[0:3000]
y_train = y_class[0:3000]
x_test = x[3000:5000]
y_test = y_class[3000:5000]

In [ ]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_test  = torch.from_numpy(x_test).float()
y_test  = torch.from_numpy(y_test).float()

#### LLAMADA DEL MODELO

In [ ]:
model = CNN().to(device)
f_loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

### Entrenamiento del modelo

In [ ]:
print("Modelo")
loss_lista_train, loss_lista_test = train(model,f_loss, optimizer, 250, x_train,y_train, x_test, y_test, 300)

### Evaluacion de resultados

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics



x_test= x_test.to(device)

predicted =   model(x_test)

predicted = predicted.cpu()

predictions_label = [1 if predicted[i] >= 0.5 else 0 for i in range(len(predicted))]

y_test = y_test.cpu()

real = [1 if y_test[i] >= 0.5 else 0 for i in range(len(y_test))]

confusion_matrix = metrics.confusion_matrix(real, predictions_label )

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()
Accuracy = metrics.accuracy_score(real, predictions_label )
print("Accuracy",Accuracy)
Precision = metrics.precision_score(real, predictions_label )
print("Precision",Precision)
Sensitivity_recall = metrics.recall_score(real, predictions_label)
print("Sensitivity_recall",Sensitivity_recall)

#### MAPAS DE VALIDACION DE TRAIN Y TEST

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_lista_train, label = "train")
plt.plot(loss_lista_test, label = "test")
plt.xlabel("Epocas")
plt.ylabel("Error")
plt.legend()
plt.show()